In [2]:
from tensorflow import keras
import tensorflow as tf

D:\ANACONDA\envs\ML\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ANACONDA\envs\ML\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ANACONDA\envs\ML\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ANACONDA\envs\ML\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1ty

In [4]:
(x,y),(x_test,y_test) =tf.keras.datasets.cifar10.load_data()
print(x_test.shape,type(x_test))
print(y_test.shape,type(y_test))

(10000, 32, 32, 3) <class 'numpy.ndarray'>
(10000, 1) <class 'numpy.ndarray'>


In [6]:
vgg_layers=[
    tf.keras.layers.Conv2D(64,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.Conv2D(64,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
      #stack1
     tf.keras.layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
     tf.keras.layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
     tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
      #stack1
     tf.keras.layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
     tf.keras.layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
     tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
     tf.keras.layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
     tf.keras.layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
     tf.keras.layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
     tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
      tf.keras.layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
     tf.keras.layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
     tf.keras.layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
     tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
]

fc_layers=[
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=None)
]

In [ ]:
d

In [7]:
vgg_model=tf.keras.Sequential(vgg_layers)
vgg_model.build(input_shape=[None,32,32,3])
fc_model=tf.keras.Sequential(fc_layers)
fc_model.build(input_shape=[None,512])
optimizer = tf.keras.optimizers.Adam(lr=1e-4)

In [33]:
def feature_scale(x,y):
    x=tf.cast(x,tf.float32)/255.0
    y=tf.cast(y,tf.int32)
    print(type(x))
    return x,y

data = tf.data.Dataset.from_tensor_slices((x,y))
data = data.map(feature_scale).shuffle(10000).batch(512)
data_iter = iter(data)
print(data_iter)
samples = next(data_iter)
print(samples)

<class 'tensorflow.python.framework.ops.Tensor'>


In [31]:

variables = vgg_model.variables+fc_model.variables
for i in range(5):
    for step,(x,y) in enumerate(data):
        with tf.GradientTape() as tape:
            print(step)
            print(x.shape)
            print(y.shape)
            vgg_output=vgg_model(x)
            print(vgg_output.shape,type(vgg_output))
            fc_output = fc_model(vgg_output)
            print(fc_output.shape,type(fc_output))
            #print(fc_output)
            loss=tf.losses.categorical_crossentropy(y,fc_output)
            print(loss.shape,type(loss.shape))
            loss_mean = tf.reduce_mean(loss)
            print(loss_mean)
            print(loss_mean.shape,type(loss_mean))
        grads=tape.gradient(loss,variables)
        #print(grads)
        optimizer.apply_gradients(zip(grads,variables))
        if step%10 ==0:
            print(i,step,'loss',float(loss_mean))

<class 'tensorflow.python.framework.ops.Tensor'>
0
(512, 32, 32, 3)
(512, 1)
(512, 1, 1, 512) <class 'tensorflow.python.framework.ops.EagerTensor'>
(512, 1, 1, 10) <class 'tensorflow.python.framework.ops.EagerTensor'>
(512, 1, 512) <class 'tensorflow.python.framework.tensor_shape.TensorShape'>
tf.Tensor(5.22472e-06, shape=(), dtype=float32)
() <class 'tensorflow.python.framework.ops.EagerTensor'>
0 0 loss 5.224720098340185e-06
0
(512, 32, 32, 3)
(512, 1)
(512, 1, 1, 512) <class 'tensorflow.python.framework.ops.EagerTensor'>
(512, 1, 1, 10) <class 'tensorflow.python.framework.ops.EagerTensor'>
(512, 1, 512) <class 'tensorflow.python.framework.tensor_shape.TensorShape'>
tf.Tensor(5.22472e-06, shape=(), dtype=float32)
() <class 'tensorflow.python.framework.ops.EagerTensor'>
1 0 loss 5.224720098340185e-06
0
(512, 32, 32, 3)
(512, 1)
(512, 1, 1, 512) <class 'tensorflow.python.framework.ops.EagerTensor'>
(512, 1, 1, 10) <class 'tensorflow.python.framework.ops.EagerTensor'>
(512, 1, 512) <cla

In [34]:
for i in range(2):
    for step,(x,y) in enumerate(data):
        with tf.GradientTape() as tape:
            print(step)

0
0
0
0
0
